# Speculative Decoding: Train and Benchmark Medusa

Large Language Models (LLMs) are changing our world. However, productionizing them can be slow and expensive. Speculative decoding is a technique that can speed up LLM inference by predicting multiple future tokens in parallel. This can reduce the time required for generating text outputs. However, speculative decoding can be complex to implement. Medusa is a framework that simplifies the speculative decoding process while maintaining its benefits.

Medusa accelerates LLM text generation by adding multiple decoding heads to predict several subsequent tokens in parallel, instead of just the next token. It then uses tree attention to efficiently process multiple token candidates simultaneously and a typical acceptance scheme to select plausible continuations, resulting in about a 2x speedup in generation time. By integrating additional "Medusa heads" with the original model, it allows for efficient token generation without the need for a separate draft model. 

This blog post shows you how to train and benchmark Medusa. 

## Training Medusa

Before training our Medusa we need to better understand our data distribution. One of the most important things is to have a good dataset (with similar distribution to what will be used in production) because Medusa has a much higher hit-rate when the generation is in-domain. 

This means if you are going to train Medusa on a dataset that is very different from the data/user queries you have in production, your speedup will be minimal or non-existent. 

There are 3 different ways to select/prepare data for training Medusa:

1. **Self-distillation**: This is the easiest and most effective way to prepare data for training. You can use the same model to generate the data that you will use to train the model. Essentially, you prompt the model with a similar input to what you will use in production and the model will generate the output.
2. **User/Application data**: If you are able to collect real user queries and model outputs, you can use this data to train Medusa. 
3. **Fine-tuning data**: If you don't have access to user data, you can use the fine-tuning dataset to train Medusa.

In this blog post, we will use the fine-tuning data to train Medusa. 

The dataset or data distribution also plays a key role when evaluating/benchmarking the performance of the Medusa heads. As we learned that Medusa has a much higher hit-rate when the generation is in-domain, it is important to evaluate the Medusa heads on the same data distribution that will be used in production or training. I

Okay lets get started. 🚀 We will use the [original implementation of Medusa](https://github.com/FasterDecoding/Medusa).

In [ ]:
git clone https://github.com/FasterDecoding/Medusa.git
cd Medusa
pip install -e .